In [42]:
from sys import path
import os 
from extraction import extraction
from load import load
import math
from transformation import transformations
import pycountry

path.append(os.path.realpath("../"))

In [43]:
def extraction_api(path):
    return extraction.get_full_data(math.ceil(extraction.extract(path, 0)["total"]/100))

In [44]:
Hubspot_df = extraction_api("https://api.hubapi.com/crm/v3/objects/contacts/search")
for i in range(len(Hubspot_df)):
        if Hubspot_df.loc[i, "properties.raw_email"] != None:
            Hubspot_df.loc[i, "Email"] = transformations.exctract_email(Hubspot_df.loc[i, "properties.raw_email"])
        else:
            Hubspot_df.loc[i, "Email"] = ""
Hubspot_none_duplicates_df = transformations.Delete_duplicates(Hubspot_df)

In [4]:
test_case = Hubspot_none_duplicates_df[Hubspot_none_duplicates_df["Name"] == "Oliver Blackburn"]
test_case[["id", "Name", "Email", "properties.phone", "properties.address", "properties.country", "Industry", "properties.createdate"]]

,id,Name,Email,properties.phone,properties.address,properties.country,Industry,properties.createdate
2734,460464,Oliver Blackburn,oliver_blackburn1495449589@kideod.biz,2-432-212-485,"Fawn Street, 6509",England,;Milling;Poultry and fish;Dairy products;Anima...,2023-05-15T02:39:02.878Z


In [86]:
import pandas as pd
test_hubstpot_df = pd.read_csv("resources/Test_hubspot.csv")
test_hubstpot_df["properties.raw_email"].fillna("", inplace=True)
test_hubstpot_df

,id,properties.firstname,properties.lastname,properties.raw_email,properties.phone,properties.address,properties.country,properties.industry,properties.createdate,properties.technical_test___create_date,properties.hs_object_id
0,402335,Oliver,Blackburn,,NaN,Fawn Street 6509,England,Poultry and fish,2021-01-29 09:30:08,NaN,NaN
1,405397,Oliver,Blackburn,Oliver <oliver_blackburn1495449589@kideod.biz>...,2-432-212-485,Maple Boulevard 9759,England,Animal feeds,2021-02-03 18:45:23,NaN,NaN
2,402668,NaN,NaN,Oliver <oliver_blackburn1495449589@kideod.biz>...,NaN,Queen Caroline Drive 6882,NaN,Animal feeds,2021-04-29 20:11:53,NaN,NaN
3,401726,Oliver,Blackburn,Oliver <oliver_blackburn1495449589@kideod.biz>...,NaN,Bond Road 4019,NaN,Milling,2021-06-07 12:51:51,NaN,NaN
4,401634,Oliver,Blackburn,,NaN,Bayberry Avenue 4000,England,Dairy products,2021-08-23 18:51:48,NaN,NaN
5,403589,NaN,NaN,Oliver <oliver_blackburn1495449589@kideod.biz>...,NaN,Belgrave Lane 1564,NaN,Milling,2021-12-06 09:22:27,NaN,NaN
6,403933,Oliver,Blackburn,,NaN,Clissold Road 4950,NaN,Poultry and fish,2021-12-30 16:35:19,NaN,NaN


In [87]:
for i in range(len(test_hubstpot_df)):
    if test_hubstpot_df.loc[i, "properties.raw_email"] != "":
        test_hubstpot_df.loc[i, "Email"] = transformations.exctract_email(test_hubstpot_df.loc[i, "properties.raw_email"])
    else:
        test_hubstpot_df.loc[i, "Email"] = ""
test_hubspot_none_duplicates_df = transformations.Delete_duplicates(test_hubstpot_df)
test_hubspot_none_duplicates_df

,id,Name,Email,properties.country,properties.phone,properties.address,Industry,properties.technical_test___create_date,properties.hs_object_id,properties.createdate
0,403933,Oliver Blackburn,oliver_blackburn1495449589@kideod.biz,England,2-432-212-485,Clissold Road 4950,;Milling;Dairy products;Poultry and fish;Anima...,NaN,NaN,2021-12-30 16:35:19


In [ ]:
def transformation(dataframe):
    for i in range(len(dataframe)):
        if dataframe.loc[i, "properties.raw_email"] != None:
            dataframe.loc[i, "Email"] = transformations.exctract_email(dataframe.loc[i, "properties.raw_email"])
        else:
            dataframe.loc[i, "Email"] = ""
    dataframe = transformations.Delete_duplicates(dataframe)
    dataframe['Country'] = dataframe["properties.country"].apply(lambda x: transformations.findCountry(x)[0])
    dataframe['City'] = dataframe["properties.country"].apply(lambda x: transformations.findCountry(x)[1])
    dataframe = dataframe.replace("Éire / Ireland", "Ireland") 
    dataframe = transformations.get_phone_number(dataframe)
    return dataframe

In [ ]:
if __name__ == '__main__':
    Hubspot_df = extraction_api("https://api.hubapi.com/crm/v3/objects/contacts/search")
    fina_df = transformation(Hubspot_df)
    fina_df.to_csv('resources/output.csv', index = False)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
df_countries = fina_df.groupby('City', axis=0).count()
df_countries['Email'].plot(kind='pie',
figsize=(5, 6),
autopct='%1.1f%%', 
startangle=90, 
shadow=True,
)
plt.title('Cities from the contacts')
plt.axis('equal') 
plt.show()

In [ ]:
x_values = fina_df['Country'].unique()
y_values = fina_df['Country'].value_counts().tolist()
plt.bar(x_values, y_values)
plt.title('Countries from the contacts')
plt.show()
plt.close('all')